In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/pulmonary.csv")

In [3]:
#burns['sap2'] = 0

In [4]:
def age(item):
    temp = 0
    if (40 <= item) & (item <= 59):
        temp += 7
    elif (60 <= item) & (item <= 69):
        temp += 12
    elif (70 <= item) & (item <= 74):
        temp += 15
    elif (75 <= item) & (item <= 79):
        temp += 16
    elif (80 <= item):
        temp += 18
    return temp

In [5]:
burns['sap2_age'] = burns['age'].apply(age)

In [6]:
def heartrate(item):
    temp = 0
    if (item < 40):
        temp += 11
    elif (40 <= item) & (item <= 69):
        temp += 2
    elif (120 <= item) & (item <= 159):
        temp += 4
    elif (160 <= item):
        temp += 7
    return temp

In [7]:
burns['sap2_heartrate'] = burns['heartrate'].apply(heartrate)

In [8]:
def meanbp(item):
    temp = 0
    if (item < 70):
        temp += 13
    elif (70 <= item) & (item <= 99):
        temp += 5
    elif (200 <= item):
        temp += 2
    return temp

In [9]:
burns['sap2_meanbp'] = burns['meanbp'].apply(meanbp)

In [10]:
def temperature(item):
    temp = 0
    if (item >= 39):
        temp += 3
    return temp

In [11]:
burns['sap2_temperature'] = burns['temperature'].apply(temperature)

In [12]:
burns['GCS'] = burns['eyes'] + burns['motor'] + burns['verbal']

In [13]:
def GCS(item):
    temp = 0
    if (item >= 11) & (item <= 13):
        temp += 5
    elif (item >= 9) & (item <= 10):
        temp += 7
    elif (item >= 6) & (item <= 8):
        temp += 13
    elif (item < 6):
        temp += 26
    return temp

In [14]:
burns['sap2_GCS'] = burns['GCS'].apply(GCS)

In [15]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [16]:
apachepredvar

,apachepredvarid,patientunitstayid,sicuday,saps3day1,saps3today,saps3yesterday,gender,teachtype,region,bedcount,...,creatinine,dischargelocation,visitnumber,amilocation,day1meds,day1verbal,day1motor,day1eyes,day1pao2,day1fio2
0,1794895,141168,1,0,0,0,1,0,3,12,...,2.30,9,1,-1,0,5,6,4,-1.0,-1.0
1,2406430,141178,1,0,0,0,1,0,3,9,...,-1.00,4,1,-1,-1,-1,-1,-1,-1.0,-1.0
2,1790923,141194,1,0,0,0,0,0,3,38,...,2.51,4,1,-1,0,4,6,3,-1.0,-1.0
3,27799,141197,1,0,0,0,0,0,3,30,...,-1.00,4,1,-1,0,5,6,4,-1.0,-1.0
4,2406432,141203,1,0,0,0,1,0,3,18,...,0.56,4,1,-1,0,1,3,1,51.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,825962,3353235,1,0,0,0,0,0,3,23,...,-1.00,8,1,-1,0,5,6,4,-1.0,-1.0
171173,996721,3353237,1,0,0,0,1,0,3,11,...,1.07,4,1,-1,0,4,6,4,-1.0,-1.0
171174,917301,3353251,1,0,0,0,0,0,3,20,...,2.43,8,1,-1,0,1,1,1,80.0,100.0
171175,2418339,3353254,1,0,0,0,0,0,3,14,...,-1.00,4,1,-1,0,5,6,4,-1.0,-1.0


In [17]:
burns_merged = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns, how='right', on='patientunitstayid')

In [18]:
burns_merged['sap2_ratio'] = 0

In [19]:
for i in range(len(burns_merged)):
    if (burns_merged.iloc[i]['ventday1'] == 1) & (burns_merged.iloc[i]['pao2'] != -1) & (burns_merged.iloc[i]['fio2'] != -1):
        
        if (burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 < 100:
            burns_merged['sap2_ratio'].iloc[i] = 11
        elif ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 100) & ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 199):
            burns_merged['sap2_ratio'].iloc[i] = 9
        elif ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 200):
            burns_merged['sap2_ratio'].iloc[i] = 6

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [20]:
def BUN(item):
    temp = 0
    if (item >= 28) & (item <= 83):
        temp += 6
    elif (item >= 84):
        temp += 10
    return temp

In [21]:
#burns_1 = pd.merge(burns_merged[['patientunitstayid','sap2_ratio']], burns, how='right', on='patientunitstayid')

In [22]:
burns_merged['sap2_BUN'] = burns_merged['BUN'].apply(BUN)

In [23]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [24]:
burns_merged_1 = pd.merge(apacheapsdvar[['patientunitstayid','urine']], burns_merged, how='right', on='patientunitstayid')

In [25]:
def urine(item):
    temp = 0
    if (item <500) & (item != -1):
        temp += 11
    elif (item >= 500) & (item <= 999):
        temp += 4
    return temp

In [26]:
burns_merged_1['sap2_urine'] = burns_merged_1['urine'].apply(urine)

In [27]:
def sodium(item):
    temp = 0
    if (item <125):
        temp += 5
    elif (item >= 145):
        temp += 1
    return temp

In [28]:
burns_merged_1['sap2_sodium'] = burns_merged_1['sodium'].apply(sodium)

In [29]:
def potassium(item):
    temp = 0
    if (item < 3):
        temp += 3
    elif (item >= 5):
        temp += 3
    return temp

In [30]:
burns_merged_1['sap2_potassium'] = burns_merged_1['potassium'].apply(potassium)

In [31]:
def bicarbonate(item):
    temp = 0
    if (item < 15):
        temp += 6
    elif (item >= 15) & (item <= 19):
        temp += 3
    return temp

In [32]:
burns_merged_1['sap2_bicarbonate'] = burns_merged_1['bicarbonate'].apply(bicarbonate)

In [33]:
burns_merged_2 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged_1, how='right', on='patientunitstayid')

In [34]:
def bilirubin(item):
    temp = 0
    if (item >= 4) & (item <= 5.9):
        temp += 4
    elif (item >= 6):
        temp += 9
    return temp

In [35]:
burns_merged_2['sap2_bilirubin'] = burns_merged_2['bilirubin'].apply(bilirubin)

In [36]:
def WBC(item):
    temp = 0
    if (item < 1):
        temp += 12
    elif (item >= 20):
        temp += 3
    return temp

In [37]:
burns_merged_2['sap2_wbc'] = burns_merged_2['WBC x 1000'].apply(WBC)

In [38]:
burns_merged_3 = pd.merge(apachepredvar[['patientunitstayid','metastaticcancer']], burns_merged_2, how='right', on='patientunitstayid')

In [39]:
burns_merged_4 = pd.merge(apachepredvar[['patientunitstayid','leukemia']], burns_merged_3, how='right', on='patientunitstayid')

In [40]:
def metastaticcancer(item):
    temp = 0
    if (item == 1):
        temp += 9
    return temp

In [41]:
burns_merged_4['sap2_metastaticcancer'] = burns_merged_4['metastaticcancer'].apply(metastaticcancer)

In [42]:
def leukemia(item):
    temp = 0
    if (item == 1):
        temp += 10
    return temp

In [43]:
def lymphoma(item):
    temp = 0
    if (item == 1):
        temp += 10
    return temp

In [44]:
def aids(item):
    temp = 0
    if (item == 1):
        temp += 17
    return temp

In [45]:
burns_merged_4['sap2_leukemia'] = burns_merged_4['leukemia'].apply(leukemia)

In [46]:
burns_merged_4['sap2_lymphoma'] = burns_merged_4['lymphoma'].apply(lymphoma)

In [47]:
burns_merged_4['sap2_aids'] = burns_merged_4['aids'].apply(aids)

In [48]:
burns_merged_4['unittype'].value_counts()

Med-Surg ICU    20080
MICU             3856
Cardiac ICU      2380
SICU             1958
CCU-CTICU        1910
CSICU             984
Neuro ICU         862
CTICU             801
Name: unittype, dtype: int64

In [49]:
def admit(item):
    temp = 0
    if (item == "Med-Surg ICU") | (item == "MICU"):
        temp += 6
    elif (item == "SICU") | (item == "CSICU"):
        temp += 8
    return temp

In [50]:
burns_merged_4['sap2_admit'] = burns_merged_4['unittype'].apply(admit)

In [51]:
burns_merged_4

,patientunitstayid,leukemia,metastaticcancer,bilirubin,urine,pao2,fio2,gender,age,ethnicity,...,sap2_sodium,sap2_potassium,sap2_bicarbonate,sap2_bilirubin,sap2_wbc,sap2_metastaticcancer,sap2_leukemia,sap2_lymphoma,sap2_aids,sap2_admit
0,141337,0,0,-1.0,-1.0000,-1.0,-1.0,Female,72.0,Hispanic,...,0,0,0,0,0,0,0,0,0,6
1,141436,0,0,-1.0,-1.0000,141.0,40.0,Male,46.0,Hispanic,...,0,0,0,0,0,0,0,0,0,0
2,141462,0,0,-1.0,-1.0000,-1.0,-1.0,Male,80.0,Caucasian,...,0,0,0,0,0,0,0,0,0,6
3,141515,0,0,1.0,-1.0000,85.0,70.0,Female,68.0,Caucasian,...,0,0,0,0,0,0,0,0,0,0
4,141585,0,0,-1.0,-1.0000,76.0,50.0,Male,63.0,Caucasian,...,0,0,0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32826,3353198,0,0,0.5,670.0320,52.0,50.0,Female,66.0,Caucasian,...,1,0,0,0,0,0,0,0,0,6
32827,3353200,0,0,-1.0,1483.9200,70.0,30.0,Female,66.0,Caucasian,...,0,0,0,0,0,0,0,0,0,0
32828,3353201,0,0,-1.0,2217.9744,329.0,100.0,Female,66.0,Caucasian,...,1,0,0,0,0,0,0,0,0,0
32829,3353226,0,0,-1.0,-1.0000,-1.0,-1.0,Female,79.0,African American,...,0,3,0,0,0,0,0,0,0,0


In [52]:
burns_merged_4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32831 entries, 0 to 32830
Data columns (total 81 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   patientunitstayid      32831 non-null  int64  
 1   leukemia               32831 non-null  int64  
 2   metastaticcancer       32831 non-null  int64  
 3   bilirubin              32831 non-null  float64
 4   urine                  32831 non-null  float64
 5   pao2                   32831 non-null  float64
 6   fio2                   32831 non-null  float64
 7   gender                 32831 non-null  object 
 8   age                    32831 non-null  float64
 9   ethnicity              32831 non-null  object 
 10  apacheadmissiondx      32831 non-null  object 
 11  admissionheight        32831 non-null  float64
 12  hospitaladmitoffset    32831 non-null  float64
 13  unittype               32831 non-null  object 
 14  unitadmitsource        32831 non-null  object 
 15  un

In [53]:
burns_merged_4['sap2'] = burns_merged_4[['sap2_age', 'sap2_heartrate', 'sap2_meanbp', 'sap2_temperature',
       'sap2_GCS', 'sap2_ratio', 'sap2_BUN', 'sap2_urine', 'sap2_sodium',
       'sap2_potassium', 'sap2_bicarbonate', 'sap2_bilirubin', 'sap2_wbc',
       'sap2_metastaticcancer', 'sap2_leukemia', 'sap2_lymphoma', 'sap2_aids',
       'sap2_admit']].sum(axis=1)

In [54]:
burns_merged_4['sap2_logit'] = -7.7631+0.0737*burns_merged_4['sap2']+0.9971*np.log(burns_merged_4['sap2']+1)

In [55]:
burns_merged_4['sap2_mortality'] = np.exp(burns_merged_4['sap2_logit'])/(1+np.exp(burns_merged_4['sap2_logit']))

In [56]:
burns_merged_4['sap2_mortality']

0        0.071716
1        0.071716
2        0.196366
3        0.437547
4        0.166523
           ...   
32826    0.753243
32827    0.196366
32828    0.285486
32829    0.128048
32830    0.087706
Name: sap2_mortality, Length: 32831, dtype: float64

In [57]:
burns_merged_4['actualicumortality'].value_counts()

ALIVE      29702
EXPIRED     3129
Name: actualicumortality, dtype: int64

In [58]:
burns_merged_4['actualicumortality_label'] = burns_merged_4['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [59]:
burns_merged_4['actualicumortality_label'].values

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [60]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_4['actualicumortality_label'].values, burns_merged_4['sap2_mortality'].values, pos_label=1)

In [61]:
metrics.auc(fpr, tpr)

0.7803253502744283